# 02 — DATASHEET DRILL: 74HC00 Quad NAND Gate

**Module 06: Integrated Circuits**

---

This notebook is a **complete walkthrough** of a real IC datasheet. We will go section by section through the 74HC00 (quad 2-input NAND gate) datasheet, understanding every specification and calculating what the numbers mean for real circuit design.

By the end, you will:
- Read any digital IC datasheet with confidence
- Understand absolute maximum ratings vs. recommended operating conditions
- Interpret DC and AC electrical characteristics
- Calculate total power consumption at a given operating frequency
- Determine whether the 74HC00 can drive a specific load (like an LED)
- Know the rules for unused inputs and latch-up prevention

**Grab a copy of the 74HC00 datasheet** (from NXP, TI, or ON Semi -- they are all equivalent) and follow along.

## Section 1 — Part Overview

The first page of any datasheet tells you what the part does:

```
74HC00 / 74HCT00
Quad 2-input NAND gate

- High-Speed Si-gate CMOS device
- Complies with JEDEC standard No. 7A
- Input levels: CMOS (74HC00) or TTL-compatible (74HCT00)
- ESD protection: HBM > 2000V, CDM > 1000V
```

### Package: DIP-14 (for breadboarding) or SOIC-14 (for PCBs)

```
        +---\/---+
   1A  -| 1   14 |- VCC
   1B  -| 2   13 |- 4B
   1Y  -| 3   12 |- 4A
   2A  -| 4   11 |- 4Y
   2B  -| 5   10 |- 3B
   2Y  -| 6    9 |- 3A
  GND  -| 7    8 |- 3Y
        +--------+
```

**Four independent NAND gates** in one 14-pin package.
- VCC on pin 14, GND on pin 7 (standard for 14-pin 74xx series)
- Each gate has 2 inputs (A, B) and 1 output (Y)
- Y = NOT(A AND B)

### Function Table (Truth Table)

| A | B | Y |
|---|---|---|
| L | L | H |
| L | H | H |
| H | L | H |
| H | H | L |

H = HIGH level, L = LOW level. This is the NAND function: output is LOW only when both inputs are HIGH.

## Section 2 — Absolute Maximum Ratings

This section lists conditions that will **damage or destroy** the IC. Never exceed these values, even briefly.

```
ABSOLUTE MAXIMUM RATINGS (damage beyond these)

Parameter                    Symbol    Value
Supply voltage               VCC       -0.5 to +7.0 V
Input voltage                V_I       -0.5 to VCC + 0.5 V
Output voltage               V_O       -0.5 to VCC + 0.5 V
Input/output clamp current   I_IK      +/- 20 mA
Output current (source)      I_OH      -25 mA
Output current (sink)        I_OL      25 mA
Supply current               I_CC      50 mA
Power dissipation (package)  P_D       500 mW
Storage temperature          T_stg     -65 to +150 C
Operating temperature        T_op      -40 to +125 C
```

### Key warnings:

1. **VCC max = 7.0V**: Applying 9V or 12V will destroy the IC instantly. Always check your power supply.

2. **Input voltage: -0.5V to VCC + 0.5V**: Inputs must not go more than 0.5V below GND or above VCC. This is critical when mixing voltage levels. A 5V signal into a 3.3V-powered 74HC00 violates this limit (5.0V > 3.3V + 0.5V = 3.8V).

3. **Output current: 25 mA max**: The output transistors can only handle 25 mA before damage. This limits what you can directly drive.

4. **These are NOT operating conditions**. The chip may work at 6.5V VCC but its specifications are not guaranteed. The recommended operating conditions (next section) are where the chip is guaranteed to work correctly.

## Section 3 — Recommended Operating Conditions

These are the conditions under which the datasheet guarantees all specifications:

```
RECOMMENDED OPERATING CONDITIONS

Parameter             Symbol   74HC00              74HCT00
Supply voltage        VCC      2.0 to 6.0 V       4.5 to 5.5 V
Input voltage         V_I      0 to VCC            0 to VCC
Output current        I_OH     -4 mA (at 4.5V)     -4 mA
                      I_OL     4 mA (at 4.5V)      4 mA
Operating temperature T_A      -40 to +125 C       -40 to +125 C
Input rise/fall time  t_r/t_f  <= 500 ns (at 4.5V) <= 500 ns
```

### Key observations:

1. **74HC VCC range: 2.0 to 6.0V** -- this is extremely flexible! You can run it at 3.3V, 5V, or anything in between. The HCT version is locked to 5V (+/- 10%).

2. **Output current: +/- 4 mA** -- this is the recommended operating current, not the absolute max (25 mA). At 4 mA, the output voltage specs are guaranteed. At higher currents, V_OH drops and V_OL rises.

3. **Input rise/fall time: <= 500 ns** -- if input edges are slower than 500 ns, the chip may oscillate or draw excess current during the transition. This is because both MOSFETS are partially on during slow transitions.

## Section 4 — DC Electrical Characteristics

This is where the real specifications live. These tell you what voltages and currents to expect under static (DC) conditions.

```
DC ELECTRICAL CHARACTERISTICS (74HC00)
T_A = 25C unless otherwise noted

Parameter    Conditions          VCC    Min    Typ    Max    Unit
V_IH         Input HIGH          2.0V   1.5    -      -      V
                                 4.5V   3.15   -      -      V
                                 6.0V   4.2    -      -      V

V_IL         Input LOW           2.0V   -      -      0.5    V
                                 4.5V   -      -      1.35   V
                                 6.0V   -      -      1.8    V

V_OH         Output HIGH         2.0V   1.9    2.0    -      V
             (I_OH = -20 uA)     4.5V   4.4    4.5    -      V
                                 6.0V   5.9    6.0    -      V

V_OH         Output HIGH         4.5V   3.98   4.32   -      V
             (I_OH = -4 mA)

V_OL         Output LOW          2.0V   -      0      0.1    V
             (I_OL = 20 uA)      4.5V   -      0      0.1    V
                                 6.0V   -      0      0.1    V

V_OL         Output LOW          4.5V   -      0.15   0.33   V
             (I_OL = 4 mA)

I_I          Input current       6.0V   -      -      +/-1   uA

I_CC         Quiescent supply    6.0V   -      -      4      uA
             current (per pkg)
```

### Critical details to notice:

1. **Input thresholds scale with VCC**: V_IH = ~0.7 * VCC, V_IL = ~0.3 * VCC. This is characteristic of CMOS.

2. **Output voltages depend on load current**: At 20 uA (light load), V_OH = 4.4V min. At 4 mA (heavier load), V_OH drops to 3.98V min. The output MOSFET has finite on-resistance, causing this voltage drop.

3. **I_I = +/- 1 uA**: Input current is essentially zero (MOSFET gate). This is why CMOS has virtually unlimited fan-out in DC terms.

4. **I_CC = 4 uA max**: The entire 4-gate IC draws at most 4 microamps when not switching. At 5V, that is 20 microwatts. Extraordinary.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Noise margin calculation at different VCC levels from datasheet values

VCC_vals = np.array([2.0, 4.5, 6.0])

# Datasheet values
VIH = np.array([1.5, 3.15, 4.2])    # Input HIGH threshold (min)
VIL = np.array([0.5, 1.35, 1.8])    # Input LOW threshold (max)
VOH = np.array([1.9, 4.4, 5.9])     # Output HIGH (min, at -20 uA)
VOL = np.array([0.1, 0.1, 0.1])     # Output LOW (max, at 20 uA)

NMH = VOH - VIH  # HIGH noise margin
NML = VIL - VOL   # LOW noise margin

print('74HC00 Noise Margins from Datasheet')
print('=' * 55)
print(f'{"VCC (V)":>8} | {"V_OH (V)":>8} | {"V_IH (V)":>8} | {"NM_H (V)":>8}')
print(f'{"":>8} | {"V_OL (V)":>8} | {"V_IL (V)":>8} | {"NM_L (V)":>8}')
print('-' * 55)
for i in range(len(VCC_vals)):
    print(f'{VCC_vals[i]:>8.1f} | {VOH[i]:>8.1f} | {VIH[i]:>8.2f} | {NMH[i]:>8.2f}')
    print(f'{"":>8} | {VOL[i]:>8.1f} | {VIL[i]:>8.2f} | {NML[i]:>8.2f}')
    print()

fig, ax = plt.subplots(figsize=(10, 6))

# Continuous interpolation
VCC_cont = np.linspace(2.0, 6.0, 100)
NMH_cont = np.interp(VCC_cont, VCC_vals, NMH)
NML_cont = np.interp(VCC_cont, VCC_vals, NML)

ax.plot(VCC_cont, NMH_cont, 'g-', linewidth=2.5, label='NM_H (HIGH noise margin)')
ax.plot(VCC_cont, NML_cont, 'r-', linewidth=2.5, label='NM_L (LOW noise margin)')
ax.plot(VCC_vals, NMH, 'go', markersize=10)
ax.plot(VCC_vals, NML, 'ro', markersize=10)

for i in range(len(VCC_vals)):
    ax.annotate(f'{NMH[i]:.2f}V', xy=(VCC_vals[i], NMH[i]),
                xytext=(VCC_vals[i] + 0.1, NMH[i] + 0.05), fontsize=10, color='green')
    ax.annotate(f'{NML[i]:.2f}V', xy=(VCC_vals[i], NML[i]),
                xytext=(VCC_vals[i] + 0.1, NML[i] + 0.05), fontsize=10, color='red')

# TTL reference
ax.axhline(y=0.4, color='blue', linestyle='--', alpha=0.5, label='TTL noise margin (0.4V)')

ax.set_xlabel('Supply Voltage VCC (V)', fontsize=12)
ax.set_ylabel('Noise Margin (V)', fontsize=12)
ax.set_title('74HC00 Noise Margins vs Supply Voltage\n(from Datasheet Values)', fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 2.0)

plt.tight_layout()
plt.show()

print('Even at the lowest VCC (2.0V), the 74HC00 noise margin (0.4V) matches TTL at 5V.')
print('At 6.0V, CMOS noise margins are 4x better than TTL.')

## Section 5 — AC Electrical Characteristics

The AC section tells you how fast the chip can switch:

```
AC ELECTRICAL CHARACTERISTICS (74HC00)
C_L = 50 pF, T_A = 25C

Parameter         Conditions    VCC    Min   Typ   Max   Unit
t_PHL             --            2.0V   --    19    95    ns
(propagation      --            4.5V   --    8     15    ns
 delay, H-to-L)   --            6.0V   --    7     13    ns

t_PLH             --            2.0V   --    19    95    ns
(propagation      --            4.5V   --    8     15    ns
 delay, L-to-H)   --            6.0V   --    7     13    ns

t_THL             --            2.0V   --    19    95    ns
(transition       --            4.5V   --    7     15    ns
 time, H-to-L)    --            6.0V   --    6     13    ns

t_TLH             --            2.0V   --    19    95    ns
(transition       --            4.5V   --    7     15    ns
 time, L-to-H)    --            6.0V   --    6     13    ns

C_PD              Power dissipation     --    22    --    pF
                  capacitance (per gate)

C_I               Input capacitance     --    3.5   --    pF
```

### Key observations:

1. **Propagation delay scales with VCC**: 19 ns at 2V, 8 ns at 4.5V, 7 ns at 6V. Higher voltage = faster.

2. **Min vs Max**: Typical delay is 8 ns but worst-case is 15 ns at 4.5V. Always design for the worst case (max value).

3. **C_PD = 22 pF**: This is the key number for power calculation. It represents the effective capacitance that gets charged/discharged each switching cycle.

4. **C_I = 3.5 pF**: Each input adds 3.5 pF to the load of the driving gate. 10 inputs = 35 pF additional load.

5. **Test condition: C_L = 50 pF**: All timing specs assume a 50 pF load capacitance. Your actual circuit may have more or less, changing the actual delay.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Propagation delay vs supply voltage (datasheet values)

VCC_data = np.array([2.0, 4.5, 6.0])

tPD_typ = np.array([19, 8, 7])    # typical (ns)
tPD_max = np.array([95, 15, 13])  # maximum (ns)

# Smooth interpolation
VCC_smooth = np.linspace(2.0, 6.0, 100)
tPD_typ_smooth = np.interp(VCC_smooth, VCC_data, tPD_typ)
tPD_max_smooth = np.interp(VCC_smooth, VCC_data, tPD_max)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Propagation delay
ax1.plot(VCC_smooth, tPD_typ_smooth, 'b-', linewidth=2.5, label='Typical')
ax1.plot(VCC_smooth, tPD_max_smooth, 'r--', linewidth=2, label='Maximum (worst case)')
ax1.plot(VCC_data, tPD_typ, 'bo', markersize=8)
ax1.plot(VCC_data, tPD_max, 'rs', markersize=8)

ax1.fill_between(VCC_smooth, tPD_typ_smooth, tPD_max_smooth, alpha=0.15, color='gray',
                 label='Operating range')

for v, tt, tm in zip(VCC_data, tPD_typ, tPD_max):
    ax1.annotate(f'typ: {tt} ns\nmax: {tm} ns', xy=(v, tt),
                xytext=(v + 0.15, tt + 8), fontsize=9,
                arrowprops=dict(arrowstyle='->', lw=1))

ax1.set_xlabel('Supply Voltage VCC (V)', fontsize=12)
ax1.set_ylabel('Propagation Delay (ns)', fontsize=12)
ax1.set_title('74HC00 Propagation Delay vs VCC\n(Datasheet Values, C_L = 50 pF)', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 110)

# Maximum toggle frequency
fmax_typ = 1 / (2 * tPD_typ_smooth * 1e-9) / 1e6  # MHz
fmax_max = 1 / (2 * tPD_max_smooth * 1e-9) / 1e6   # MHz (guaranteed)

ax2.plot(VCC_smooth, fmax_typ, 'b-', linewidth=2.5, label='Typical max freq')
ax2.plot(VCC_smooth, fmax_max, 'r--', linewidth=2, label='Guaranteed max freq')

ax2.set_xlabel('Supply Voltage VCC (V)', fontsize=12)
ax2.set_ylabel('Maximum Frequency (MHz)', fontsize=12)
ax2.set_title('74HC00 Maximum Toggle Frequency\n(Calculated from Propagation Delay)', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('Design rule: ALWAYS use the MAX (worst-case) delay for timing calculations.')
print('The typical values are for information only -- your specific chip may be slower.')
print(f'\nAt VCC = 4.5V, guaranteed max frequency: {1/(2*15e-9)/1e6:.1f} MHz')
print(f'At VCC = 4.5V, typical max frequency: {1/(2*8e-9)/1e6:.1f} MHz')

## Section 6 — Power Consumption Calculation

The total power consumed by a CMOS IC has two components:

### Static Power (quiescent)
```
P_static = I_CC * VCC
P_static = 4 uA * 5V = 20 uW  (for the entire package)
```
This is essentially zero and can usually be ignored.

### Dynamic Power (switching)
```
P_dynamic = C_PD * VCC^2 * f_sw    (per gate)

Where:
  C_PD = 22 pF (from datasheet)
  VCC = supply voltage
  f_sw = switching frequency of that gate
```

### Total Power
```
P_total = P_static + SUM(C_PD * VCC^2 * f_sw) for each gate
```

If you also have external load capacitance C_L:
```
P_total = P_static + SUM((C_PD + C_L) * VCC^2 * f_sw) for each gate
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Worked example: calculate 74HC00 power consumption

print('WORKED EXAMPLE: 74HC00 Power Consumption')
print('=' * 55)
print()

# Given:
VCC = 5.0       # V
C_PD = 22e-12   # F (22 pF, from datasheet)
I_CC = 4e-6     # A (4 uA, max quiescent current)

# Scenario: 2 gates switching at 10 MHz, 2 gates at 1 MHz
# Each gate has 15 pF external load
C_L = 15e-12    # F (15 pF external load per gate)
C_total = C_PD + C_L  # total capacitance per gate

print(f'Supply voltage: VCC = {VCC} V')
print(f'Datasheet C_PD = {C_PD*1e12:.0f} pF per gate')
print(f'External load: C_L = {C_L*1e12:.0f} pF per gate')
print(f'Total capacitance per gate: {C_total*1e12:.0f} pF')
print()

# Static power
P_static = I_CC * VCC
print(f'1. Static power: P_static = I_CC x VCC = {I_CC*1e6:.0f} uA x {VCC} V = {P_static*1e6:.0f} uW')

# Dynamic power for each gate
f1, f2 = 10e6, 1e6  # Hz
n1, n2 = 2, 2         # number of gates at each frequency

P_gate_10MHz = C_total * VCC**2 * f1
P_gate_1MHz = C_total * VCC**2 * f2

print(f'\n2. Dynamic power per gate at 10 MHz:')
print(f'   P = (C_PD + C_L) x VCC^2 x f')
print(f'   P = {C_total*1e12:.0f} pF x ({VCC} V)^2 x {f1/1e6:.0f} MHz')
print(f'   P = {P_gate_10MHz*1e3:.3f} mW per gate')

print(f'\n3. Dynamic power per gate at 1 MHz:')
print(f'   P = {C_total*1e12:.0f} pF x ({VCC} V)^2 x {f2/1e6:.0f} MHz')
print(f'   P = {P_gate_1MHz*1e3:.4f} mW per gate')

P_total = P_static + n1 * P_gate_10MHz + n2 * P_gate_1MHz
print(f'\n4. Total power for the package:')
print(f'   P_total = P_static + {n1} x P_10MHz + {n2} x P_1MHz')
print(f'   P_total = {P_static*1e6:.0f} uW + {n1} x {P_gate_10MHz*1e3:.3f} mW + {n2} x {P_gate_1MHz*1e3:.4f} mW')
print(f'   P_total = {P_total*1e3:.3f} mW')
print(f'\n   Total supply current: I = P/VCC = {P_total/VCC*1e3:.3f} mA')

# Compare: same gates at 3.3V
VCC_33 = 3.3
P_total_33 = I_CC * VCC_33 + n1 * C_total * VCC_33**2 * f1 + n2 * C_total * VCC_33**2 * f2
print(f'\n5. Same circuit at VCC = 3.3V:')
print(f'   P_total = {P_total_33*1e3:.3f} mW')
print(f'   Power reduction: {(1 - P_total_33/P_total)*100:.1f}%')
print(f'   (Ratio = (3.3/5.0)^2 = {(3.3/5.0)**2:.3f} = {(3.3/5.0)**2*100:.1f}% of 5V power)')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Power consumption vs frequency at different VCC levels

freq = np.logspace(0, 8, 200)  # 1 Hz to 100 MHz
C_PD = 22e-12  # F
C_L = 15e-12   # F external load
C_total = C_PD + C_L
I_CC = 4e-6    # A

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Power vs frequency at different VCC
for VCC, color, style in [(2.0, 'green', '-'), (3.3, 'blue', '-'), (5.0, 'red', '-')]:
    # Total power for 4 gates all switching at same frequency
    P_static = I_CC * VCC
    P_dynamic = 4 * C_total * VCC**2 * freq
    P_total = P_static + P_dynamic
    
    ax1.loglog(freq, P_total * 1e3, color=color, linestyle=style, linewidth=2.5,
              label=f'VCC = {VCC}V')

ax1.set_xlabel('Switching Frequency (Hz)', fontsize=12)
ax1.set_ylabel('Total Package Power (mW)', fontsize=12)
ax1.set_title('74HC00 Power vs Frequency\n(4 gates switching, C_L = 15 pF each)',
              fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, which='both')
ax1.set_ylim(1e-5, 100)

# Power vs VCC at fixed frequency
VCC_sweep = np.linspace(2.0, 6.0, 100)
for f_val, color, label in [(1e3, 'green', '1 kHz'), (1e6, 'blue', '1 MHz'), (1e7, 'red', '10 MHz')]:
    P_static = I_CC * VCC_sweep
    P_dynamic = 4 * C_total * VCC_sweep**2 * f_val
    P_total = P_static + P_dynamic
    ax2.plot(VCC_sweep, P_total * 1e3, color=color, linewidth=2.5, label=f'f = {label}')

ax2.set_xlabel('Supply Voltage VCC (V)', fontsize=12)
ax2.set_ylabel('Total Package Power (mW)', fontsize=12)
ax2.set_title('74HC00 Power vs Supply Voltage\n(4 gates switching)',
              fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('Key insight: Power scales as VCC^2 -- reducing VCC from 5V to 3.3V saves 56% power.')
print('This is why the entire industry moved from 5V to 3.3V to 1.8V to 1.2V logic.')

## Section 7 — Output Drive: Can the 74HC00 Drive an LED?

A common beginner question: can I connect an LED directly to a logic gate output?

### What the datasheet says

```
Recommended output current:
  I_OH = -4 mA  (source current, output HIGH)
  I_OL = 4 mA   (sink current, output LOW)

Absolute maximum:
  I_OH = -25 mA
  I_OL = 25 mA
```

### LED requirements

A typical LED needs:
- Forward voltage: ~2.0V (red), ~3.2V (blue/white)
- Forward current: 10-20 mA for full brightness, 2-5 mA for dim but visible

### Analysis: LED active-LOW (sinking current)

```
    VCC (5V)
     |
     +--- [R] --- [LED] --- [74HC00 output]
                              (output LOW = LED ON)
```

When output is LOW, current flows from VCC through R and LED into the output pin.
- Available voltage: VCC - V_LED - V_OL = 5.0 - 2.0 - 0.33 = 2.67V
- For I = 4 mA: R = 2.67V / 4 mA = **668 ohm** (use 680 ohm)
- For I = 10 mA: R = 2.67V / 10 mA = 267 ohm -- but this exceeds recommended I_OL!

### Verdict

The 74HC00 can drive an LED at **reduced brightness** (4 mA) within recommended specs. For full brightness (20 mA), you need a buffer transistor.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Design exercise: LED driving from 74HC00

print('DESIGN EXERCISE: Driving an LED from 74HC00')
print('=' * 50)
print()

VCC = 5.0
V_LED_red = 2.0    # typical red LED forward voltage
V_LED_blue = 3.2   # typical blue LED forward voltage

# Output voltage under load (from datasheet)
VOL_4mA = 0.33     # V_OL at I_OL = 4 mA (max)
VOH_4mA = 3.98     # V_OH at I_OH = -4 mA (min)

# Method 1: Active-LOW (sink current through output)
print('Method 1: Active-LOW (LED on when output = LOW)')
print('  Circuit: VCC -> R -> LED -> output pin')
print()

for name, V_LED in [('Red LED', V_LED_red), ('Blue LED', V_LED_blue)]:
    V_available = VCC - V_LED - VOL_4mA
    I_target = 4e-3  # 4 mA (recommended max)
    R_needed = V_available / I_target
    # Standard resistor value
    R_standard = int(np.ceil(R_needed / 10) * 10)  # round up to nearest 10
    I_actual = V_available / R_standard
    P_resistor = I_actual**2 * R_standard
    
    print(f'  {name} (V_f = {V_LED}V):')
    print(f'    V_available = {VCC} - {V_LED} - {VOL_4mA} = {V_available:.2f}V')
    print(f'    R = {V_available:.2f}V / {I_target*1e3:.0f}mA = {R_needed:.0f} ohm (use {R_standard} ohm)')
    print(f'    Actual current: {I_actual*1e3:.1f} mA')
    if V_available < 0:
        print(f'    WARNING: Not enough voltage! Cannot drive this LED.')
    print()

# Method 2: Active-HIGH (source current from output)
print('Method 2: Active-HIGH (LED on when output = HIGH)')
print('  Circuit: output pin -> R -> LED -> GND')
print()

for name, V_LED in [('Red LED', V_LED_red), ('Blue LED', V_LED_blue)]:
    V_available = VOH_4mA - V_LED
    I_target = 4e-3
    if V_available > 0:
        R_needed = V_available / I_target
        R_standard = int(np.ceil(R_needed / 10) * 10)
        I_actual = V_available / R_standard
        print(f'  {name} (V_f = {V_LED}V):')
        print(f'    V_available = {VOH_4mA} - {V_LED} = {V_available:.2f}V')
        print(f'    R = {V_available:.2f}V / {I_target*1e3:.0f}mA = {R_needed:.0f} ohm (use {R_standard} ohm)')
        print(f'    Actual current: {I_actual*1e3:.1f} mA')
    else:
        print(f'  {name}: V_OH ({VOH_4mA}V) < V_LED ({V_LED}V) -- CANNOT DRIVE!')
    print()

print('SUMMARY:')
print('  - Red LED: works either way at ~4 mA (dim but visible)')
print('  - Blue LED: works active-LOW only; active-HIGH has marginal voltage')
print('  - For full brightness (20 mA), use an NPN transistor as a buffer')

## Section 8 — Unused Inputs and Latch-Up

### Unused Inputs: Tie Them!

**Never leave a CMOS input floating** (unconnected). Here is why:

1. A floating CMOS input has extremely high impedance (~10^12 ohm)
2. Stray electromagnetic fields easily induce voltages on the pin
3. The voltage drifts into the transition region (between V_IL and V_IH)
4. In this region, both PMOS and NMOS are partially on
5. This creates a DC path from VCC to GND --> **excessive power consumption**
6. The gate may also **oscillate**, causing noise and interference

### How to tie unused inputs:

```
For unused NAND inputs:  Tie to VCC (input = HIGH, output = other input inverted)
                    or:  Tie to GND (input = LOW, output = always HIGH)

For unused gates:        Tie ALL inputs to VCC or GND
                         (Leave output unconnected -- that is OK)
```

### Latch-Up: What It Is

CMOS ICs have parasitic thyristor (SCR) structures formed by the N-well and P-substrate:

```
  VCC ----[Rwell]----+----[parasitic NPN]----+
                     |                        |
                [parasitic PNP]               |
                     |                        |
  GND ----[Rsub]-----+------------------------+
```

If an input or output voltage exceeds VCC or goes below GND (even briefly), it can trigger this parasitic thyristor, creating a low-resistance path from VCC to GND. This draws massive current and can **destroy the IC** within milliseconds.

### How to prevent latch-up:

1. **Never apply input signals before powering VCC** (power supply sequencing)
2. **Never exceed VCC + 0.5V on any input** (use series resistors or clamp diodes if needed)
3. **Use proper decoupling** (100 nF on every IC)
4. **Do not hot-swap** ICs while the circuit is powered

## The Material Science Why — Inside the 74HC00

The 74HC00 contains **16 MOSFETs** (4 gates x 4 transistors per NAND gate) plus ESD protection structures, all on a single silicon die about 1mm x 1mm.

### Cross-section of one NAND gate:

```
    Input A    Input B           Output
      |          |                 |
  [P1 gate] [P2 gate]    [N1 gate] [N2 gate]
      |          |          |         |
   P+ | P+    P+ | P+    N+ | N+   N+ | N+
   |  SiO2|   |  SiO2|   |  SiO2|  |  SiO2|
   +--N-well--+  +--N-well--+  +--P-substrate--+
   |___________|____________|___________________|
                P-type substrate
```

### What makes it work:

1. **SiO2 gate oxide**: ~10 nm thick insulator between gate and channel. This is what gives CMOS zero gate current and enables billions of transistors.

2. **N-well**: A doped region in the P-substrate where PMOS transistors are built. The well must be connected to VCC to reverse-bias the well-substrate junction.

3. **ESD protection diodes**: Clamp diodes on every I/O pin limit voltage excursions. The datasheet's ESD rating (>2000V HBM) comes from these structures.

4. **Metal interconnects**: Aluminum or copper traces on top of the silicon connect the transistors into the NAND gate topology.

### Why the 74HC00 is "high-speed" CMOS:

The "HC" designation means the transistors are designed with shorter channel lengths and optimized gate oxide thickness compared to the original CD4000 CMOS series (which was very slow, ~100 ns). This gives 74HC its ~8 ns propagation delay while maintaining CMOS's low power advantage.

## Experiment — Datasheet Verification

Now that we have read the datasheet, let us verify its claims in the lab.

### Equipment
- 74HC00 DIP-14
- Klein MM300 multimeter
- Bench power supply (set to 5.00V)
- 100 nF ceramic capacitor
- 680 ohm resistor + red LED (optional)
- Breadboard + jumper wires

### Test 1: Verify V_OH and V_OL

1. Power up the 74HC00 (VCC=5V, GND, 100nF cap)
2. Tie unused inputs to VCC
3. Gate 1: Both inputs LOW (to GND) --> measure output voltage
   - **Expected**: V_OH >= 4.9V (datasheet says 4.4V min at -20 uA, but with no load it will be ~4.99V)
4. Gate 1: Both inputs HIGH (to VCC) --> measure output voltage
   - **Expected**: V_OL <= 0.1V (datasheet says 0.1V max at 20 uA)

### Test 2: Verify I_CC (quiescent current)

1. Set multimeter to DC current mode (mA or uA range)
2. Put the multimeter in series with VCC (between power supply and pin 14)
3. All inputs tied to VCC or GND (no switching)
4. **Expected**: I_CC < 4 uA (probably reads as ~0-1 uA on your meter)
5. Note: The Klein MM300 may not have enough resolution to read microamps accurately. If it reads "0.00 mA", that confirms the current is very small.

### Test 3: Verify output drive with LED

1. Connect: output pin --> 680 ohm --> red LED --> VCC (active-LOW)
2. Set both inputs HIGH --> output LOW --> LED should light up dimly
3. Set any input LOW --> output HIGH --> LED should be off
4. Measure the voltage across the LED and across the resistor
5. Calculate actual current: I = V_R / R

### Test 4: Floating input demo (careful!)

1. Disconnect one input of Gate 1 (leave it floating)
2. Other input tied to VCC
3. Measure the output -- it may oscillate or be unpredictable
4. Touch the floating input with your finger -- the output may change!
5. **Reconnect the input** to VCC or GND after this demo

This dramatically demonstrates why unused CMOS inputs must be tied.

## Experiment — Measuring Propagation Delay (Advanced)

If you have the **Fnirsi 2C53T oscilloscope**:

### Setup: Ring Oscillator

Chain an **odd number** of inverters (NAND gates with inputs tied together) in a ring:

```
   +--[INV 1]--[INV 2]--[INV 3]--+
   |                               |
   +-------------------------------+
         ^
         |
    Probe here (CH1)
```

This creates a **ring oscillator** that oscillates at:
```
f = 1 / (2 * N * t_PD)
```
where N = number of inverters and t_PD = propagation delay per gate.

### Procedure

1. Wire Gate 1, 2, 3 as inverters (pins 1+2 tied, pins 4+5 tied, pins 9+10 tied)
2. Connect: Gate 1 output (pin 3) to Gate 2 input (pins 4+5)
3. Connect: Gate 2 output (pin 6) to Gate 3 input (pins 9+10)
4. Connect: Gate 3 output (pin 8) back to Gate 1 input (pins 1+2) -- this closes the ring
5. Connect oscilloscope CH1 to any gate output
6. You should see an oscillating square wave!

### Calculation

Measure the oscillation frequency. Then:
```
t_PD = 1 / (2 * 3 * f_osc) = 1 / (6 * f_osc)
```

For 74HC00 at 5V, expect f_osc ~ 20 MHz, giving t_PD ~ 8 ns. This should match the datasheet.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Ring oscillator frequency prediction

N_gates = 3  # number of inverters in the ring (must be odd)

# Datasheet propagation delays at different VCC
VCC_data = np.array([2.0, 4.5, 6.0])
tPD_typ = np.array([19, 8, 7])  # ns

# Ring oscillator frequency: f = 1 / (2 * N * tPD)
f_osc = 1 / (2 * N_gates * tPD_typ * 1e-9)  # Hz

print('Ring Oscillator Predicted Frequencies')
print(f'(Using {N_gates} inverters in ring)')
print('=' * 50)
print(f'{"VCC (V)":>8} | {"tPD (ns)":>8} | {"f_osc (MHz)":>12}')
print('-' * 35)
for v, t, f in zip(VCC_data, tPD_typ, f_osc):
    print(f'{v:>8.1f} | {t:>8} | {f/1e6:>12.1f}')

# Smooth curve
VCC_smooth = np.linspace(2.0, 6.0, 100)
tPD_smooth = np.interp(VCC_smooth, VCC_data, tPD_typ)
f_osc_smooth = 1 / (2 * N_gates * tPD_smooth * 1e-9)

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(VCC_smooth, f_osc_smooth / 1e6, 'b-', linewidth=2.5)
ax.plot(VCC_data, f_osc / 1e6, 'ro', markersize=10, label='Datasheet-based prediction')

for v, f in zip(VCC_data, f_osc):
    ax.annotate(f'{f/1e6:.1f} MHz', xy=(v, f/1e6),
                xytext=(v + 0.2, f/1e6 + 1), fontsize=10,
                arrowprops=dict(arrowstyle='->', lw=1))

ax.set_xlabel('Supply Voltage VCC (V)', fontsize=12)
ax.set_ylabel('Ring Oscillator Frequency (MHz)', fontsize=12)
ax.set_title(f'Predicted Ring Oscillator Frequency\n({N_gates}-gate ring using 74HC00)',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'\nIf you measure ~{f_osc[1]/1e6:.0f} MHz at 5V, your chip is working to spec!')
print('The actual frequency may be slightly different due to:')
print('  - Wiring capacitance on the breadboard (~5-10 pF per node)')
print('  - Breadboard trace inductance')
print('  - Your specific chip being faster or slower than "typical"')

## Simulation — Build It in Falstad

### 74HC00 NAND Gate

**What to build:**
1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Build a CMOS NAND gate (two PMOS in parallel for pull-up, two NMOS in series for pull-down — as described in the previous notebooks)
3. Add switches for inputs A and B
4. Add an LED or scope on the output

**What to observe:**
- Toggle input switches and verify the NAND truth table: output LOW only when both inputs HIGH
- Watch current flow through PMOS (to VCC) and NMOS (to GND) paths for each input combination
- Notice zero static current in all stable states — this is why the datasheet's quiescent current (I_CC) is so low for CMOS

### Ring Oscillator

**What to build:**
1. Open a [new blank canvas](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Build three CMOS inverters (PMOS + NMOS pairs)
3. Connect them in a ring: output of each feeds the input of the next, with the third output feeding back to the first input
4. Right-click any node → **View in New Scope**

**What to observe:**
- The circuit oscillates on its own — no external clock needed
- Each inverter flips its output, which triggers the next, creating a self-sustaining oscillation
- The oscillation frequency depends on the propagation delay of each gate — this is directly related to the t_pd parameter on the 74HC00 datasheet
- Measure the period by counting simulation timesteps, then calculate the per-gate delay: t_pd = period / (2 × number of stages)

## Datasheet Connection — Quick Reference Card

Here is a one-page summary of everything you need from the 74HC00 datasheet for practical circuit design:

```
74HC00 QUICK REFERENCE
======================

Function: Quad 2-input NAND gate
Package:  DIP-14 (VCC=pin14, GND=pin7)
Family:   74HC (High-Speed CMOS)

POWER:
  VCC range:     2.0 - 6.0V
  I_CC (static): 4 uA max
  P_dynamic:     C_PD * VCC^2 * f  (C_PD = 22 pF/gate)

DC LEVELS (at VCC = 5V):
  V_IH >= 3.5V    (input HIGH threshold)
  V_IL <= 1.5V    (input LOW threshold)
  V_OH >= 4.4V    (output HIGH, -20 uA load)
  V_OL <= 0.1V    (output LOW, 20 uA load)
  Noise margin:   ~1.4V (both HIGH and LOW)

OUTPUT DRIVE:
  I_OH = -4 mA recommended (-25 mA absolute max)
  I_OL = 4 mA recommended (25 mA absolute max)

SPEED (at VCC = 5V, C_L = 50 pF):
  t_PD:   8 ns typical, 15 ns max
  t_rise: 7 ns typical
  f_max:  ~33 MHz (guaranteed), ~62 MHz (typical)

RULES:
  1. Always use 100 nF decoupling cap on VCC-GND
  2. Never leave inputs floating -- tie to VCC or GND
  3. Never exceed VCC + 0.5V on any input
  4. Power VCC before applying input signals
  5. Input rise/fall time must be < 500 ns
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Final summary visualization: 74HC00 operating envelope

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Propagation delay vs VCC
ax = axes[0, 0]
VCC_d = np.array([2.0, 4.5, 6.0])
tPD_typ = np.array([19, 8, 7])
tPD_max = np.array([95, 15, 13])
VCC_s = np.linspace(2.0, 6.0, 100)

ax.fill_between(VCC_s, np.interp(VCC_s, VCC_d, tPD_typ),
                np.interp(VCC_s, VCC_d, tPD_max), alpha=0.3, color='blue')
ax.plot(VCC_s, np.interp(VCC_s, VCC_d, tPD_typ), 'b-', linewidth=2, label='Typical')
ax.plot(VCC_s, np.interp(VCC_s, VCC_d, tPD_max), 'b--', linewidth=1.5, label='Maximum')
ax.set_xlabel('VCC (V)', fontsize=11)
ax.set_ylabel('Propagation Delay (ns)', fontsize=11)
ax.set_title('Propagation Delay', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# 2. Power consumption vs frequency
ax = axes[0, 1]
freq = np.logspace(0, 8, 200)
for VCC, color in [(2.0, 'green'), (3.3, 'blue'), (5.0, 'red')]:
    P = 4e-6 * VCC + 4 * 22e-12 * VCC**2 * freq
    ax.loglog(freq, P * 1e3, color=color, linewidth=2, label=f'VCC={VCC}V')
ax.set_xlabel('Frequency (Hz)', fontsize=11)
ax.set_ylabel('Package Power (mW)', fontsize=11)
ax.set_title('Power Consumption (4 gates)', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, which='both')

# 3. Noise margins vs VCC
ax = axes[1, 0]
VCC_nm = np.array([2.0, 4.5, 6.0])
VOH = np.array([1.9, 4.4, 5.9])
VOL = np.array([0.1, 0.1, 0.1])
VIH_nm = np.array([1.5, 3.15, 4.2])
VIL_nm = np.array([0.5, 1.35, 1.8])
NMH = VOH - VIH_nm
NML = VIL_nm - VOL

VCC_s2 = np.linspace(2.0, 6.0, 100)
ax.plot(VCC_s2, np.interp(VCC_s2, VCC_nm, NMH), 'g-', linewidth=2.5, label='NM_H')
ax.plot(VCC_s2, np.interp(VCC_s2, VCC_nm, NML), 'r-', linewidth=2.5, label='NM_L')
ax.plot(VCC_nm, NMH, 'go', markersize=8)
ax.plot(VCC_nm, NML, 'ro', markersize=8)
ax.axhline(y=0.4, color='gray', linestyle='--', alpha=0.5, label='TTL (ref)')
ax.set_xlabel('VCC (V)', fontsize=11)
ax.set_ylabel('Noise Margin (V)', fontsize=11)
ax.set_title('Noise Margins', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# 4. Output voltage under load
ax = axes[1, 1]
I_load = np.array([0.02, 0.1, 0.5, 1.0, 2.0, 4.0])  # mA
# Approximate from datasheet (VCC = 4.5V)
R_on_pmos = 80  # approximate on-resistance in ohms
R_on_nmos = 50
VOH_load = 4.5 - I_load * 1e-3 * R_on_pmos
VOL_load = I_load * 1e-3 * R_on_nmos

ax.plot(I_load, VOH_load, 'g-o', linewidth=2, markersize=6, label='V_OH (output HIGH)')
ax.plot(I_load, VOL_load, 'r-o', linewidth=2, markersize=6, label='V_OL (output LOW)')
ax.axhline(y=3.15, color='green', linestyle='--', alpha=0.5)
ax.axhline(y=1.35, color='red', linestyle='--', alpha=0.5)
ax.text(3.5, 3.3, 'V_IH = 3.15V', fontsize=8, color='green')
ax.text(3.5, 1.15, 'V_IL = 1.35V', fontsize=8, color='red')
ax.set_xlabel('Load Current (mA)', fontsize=11)
ax.set_ylabel('Output Voltage (V)', fontsize=11)
ax.set_title('Output Voltage vs Load\n(VCC = 4.5V)', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 5)

fig.suptitle('74HC00 Datasheet Summary', fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## Checkpoint Questions

Test your datasheet reading skills:

---

**Q1:** The 74HC00 absolute maximum VCC is 7.0V, but the recommended operating range is 2.0-6.0V. What happens if you operate at 6.5V?

<details>
<summary>Answer</summary>

At 6.5V, you are within the absolute maximum (7.0V) so the chip will not be damaged. However, you are outside the recommended operating conditions (2.0-6.0V), so the datasheet specifications are NOT guaranteed. The chip will probably work, but propagation delay, voltage thresholds, and power consumption may not meet the published specs. In a production design, this would be unacceptable.

</details>

---

**Q2:** You measure V_OH = 4.85V on your 74HC00 output (VCC = 5V, light load). Is this within spec?

<details>
<summary>Answer</summary>

Yes. The datasheet guarantees V_OH >= 4.4V (at VCC = 4.5V, -20 uA load). At VCC = 5V with light load, 4.85V is well above the minimum spec. In fact, with very light loads, V_OH approaches VCC, so ~4.95-4.99V is typical. 4.85V suggests you may have a moderate load drawing some current through the output PMOS resistance.

</details>

---

**Q3:** Calculate the total power consumption of a 74HC00 at VCC = 3.3V if Gate 1 switches at 5 MHz, Gate 2 at 1 MHz, and Gates 3 and 4 are static.

<details>
<summary>Answer</summary>

P_static = I_CC x VCC = 4 uA x 3.3V = 13.2 uW

P_gate1 = C_PD x VCC^2 x f = 22 pF x (3.3V)^2 x 5 MHz = 22e-12 x 10.89 x 5e6 = 1.198 mW

P_gate2 = 22 pF x (3.3V)^2 x 1 MHz = 0.240 mW

P_gate3 = P_gate4 = ~0 (static)

P_total = 0.013 + 1.198 + 0.240 + 0 + 0 = 1.451 mW

Supply current: I = P/VCC = 1.451 mW / 3.3V = 0.44 mA

</details>

---

**Q4:** A 3.3V microcontroller output (V_OH = 3.0V) is connected to a 74HC00 input running at VCC = 5V. Will the 74HC00 recognize this as HIGH?

<details>
<summary>Answer</summary>

No. At VCC = 5V, the 74HC00's V_IH = 3.5V. The 3.0V signal is below this threshold, so the input is in the undefined zone (between V_IL=1.5V and V_IH=3.5V). The gate output will be unpredictable. Solutions: (1) Use a 74HCT00 instead (V_IH = 2.0V for TTL-compatible inputs). (2) Run the 74HC00 at 3.3V. (3) Use a level shifter IC.

</details>

---

**Q5:** You are building a ring oscillator with 3 inverters (NAND gates wired as inverters) using the 74HC00 at VCC = 5V. The oscillation frequency you measure is 15 MHz. What propagation delay does this imply? Does it match the datasheet?

<details>
<summary>Answer</summary>

f = 1 / (2 x N x t_PD), so t_PD = 1 / (2 x 3 x 15 MHz) = 1 / (90 MHz) = 11.1 ns.

The datasheet says typical t_PD = 8 ns at VCC = 4.5V (our VCC is 5V, so it should be slightly faster). 11.1 ns is between the typical (8 ns) and maximum (15 ns) values. This is reasonable because the breadboard adds stray capacitance (~5-15 pF per node), which slows the ring oscillator compared to the datasheet test conditions (C_L = 50 pF controlled load).

</details>

---

**Q6:** Why must input rise/fall times be less than 500 ns for the 74HC00? What happens physically if the input changes too slowly?

<details>
<summary>Answer</summary>

During the transition between LOW and HIGH, the input voltage passes through the region where both the PMOS and NMOS in the output stage are partially on. This creates a temporary low-resistance path from VCC to GND, called "crowbar current" or "shoot-through current." If the input changes slowly, this current flows for a longer time, wasting power and heating the chip. With very slow transitions (~microseconds), the power dissipation can be significant enough to cause problems. The 500 ns limit ensures this through-current period is brief.

</details>

---

## Summary

| Datasheet Section | What It Tells You | Key 74HC00 Values |
|-------------------|--------------------|--------------------|
| Part overview | Function, package, pin diagram | Quad NAND, DIP-14 |
| Absolute maximum | Damage thresholds -- never exceed | VCC < 7V, I_out < 25 mA |
| Recommended operating | Guaranteed spec range | VCC = 2-6V, I_out = 4 mA |
| DC characteristics | Voltage levels, current draw | V_OH >= 4.4V, I_CC <= 4 uA |
| AC characteristics | Speed, timing | t_PD = 8 ns typ, C_PD = 22 pF |
| Power calculation | P = P_static + C_PD * VCC^2 * f | ~1 mW/gate at 10 MHz, 5V |

### The three rules of CMOS IC design:
1. **Decoupling cap on every IC** (100 nF, close to pins)
2. **Never leave inputs floating** (tie to VCC or GND)
3. **Never exceed VCC on any input** (prevents latch-up)

You can now read any digital IC datasheet. The sections and parameter names are standardized across all manufacturers and all 74xx-series parts.

**Module 06 complete!** You have gone from individual transistors to logic gates to reading IC datasheets. Next: [Module 07 -- Capstone](../module-07-capstone/) where we put it all together.